Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Here I'll implement a Tensorflow logistic regression with L2 regularization first.

In [8]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
  # Set random seed within each tf.Graph since 
  # Setting the current TensorFlow random seed affects the current default graph only.  
  tf.set_random_seed(1)
  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  # Add L2 regularization term to the loss 
  loss = loss + 0.001* tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
# Run the L2-Logit model
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))

      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 24.040695
Training accuracy: 7.3%
Validation accuracy: 9.5%
Loss at step 100: 4.866360
Training accuracy: 71.6%
Validation accuracy: 70.0%
Loss at step 200: 3.976750
Training accuracy: 74.8%
Validation accuracy: 72.8%
Loss at step 300: 3.394956
Training accuracy: 76.8%
Validation accuracy: 73.9%
Loss at step 400: 2.954026
Training accuracy: 77.8%
Validation accuracy: 74.5%
Loss at step 500: 2.602178
Training accuracy: 78.6%
Validation accuracy: 74.9%
Loss at step 600: 2.313673
Training accuracy: 79.4%
Validation accuracy: 75.5%
Loss at step 700: 2.072894
Training accuracy: 80.2%
Validation accuracy: 75.9%
Loss at step 800: 1.869541
Training accuracy: 81.0%
Validation accuracy: 76.5%
Test accuracy: 84.3%


The L2 regularization indeed improved the accuracy in both Validation (75.0% -> 76.5%) and Test (82.7% -> 84.3%) sets compared to the Logit without regularization in Assignment 2.

---

Below I'll implement the 2-layered NN with 1024 ReLUs in the hidden layer with L2-regularization

In [12]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(1)  
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # weights on the input layer, to generate values to feed into 1024 hidden ReLUs
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  # weights on the output layer
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def nn_1layer(tf_dataset):
    '''
    The decision function for NN with 1 layer of hidden units
    '''
    # input layer
    input_vals = tf.matmul(tf_dataset, weights1) + biases1
    # hidden layer
    hidden_vals = tf.nn.relu(input_vals)
    # output layer 
    logits = tf.matmul(hidden_vals, weights2) + biases2
    return logits
  
  logits = nn_1layer(tf_train_dataset)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  # Add L2 regularization term to the loss 
#   loss = loss + 0.001*(tf.nn.l2_loss(loss))
  # The l2_loss should be applied on W rather than loss?
  # Also, smaller strength of l2 regularization can improve the results
  loss = loss + 0.001*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(nn_1layer(tf_valid_dataset))
  test_prediction = tf.nn.softmax(nn_1layer(tf_test_dataset))

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 640.594604
Minibatch accuracy: 6.2%
Validation accuracy: 26.0%
Minibatch loss at step 500: 196.695541
Minibatch accuracy: 84.4%
Validation accuracy: 78.6%
Minibatch loss at step 1000: 116.928780
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 68.557335
Minibatch accuracy: 85.9%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 41.375866
Minibatch accuracy: 89.1%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.210518
Minibatch accuracy: 88.3%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 15.603547
Minibatch accuracy: 83.6%
Validation accuracy: 86.2%
Test accuracy: 93.2%


#### Conclusions for Problem1

1. `tf.nn.l2_loss(t)` should be applied to Weights because L2 regularization is defined as    
`L' = L + b * (1/2) * (W**2)`

2. A smaller strength can improve the performance for 2-layered NN.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [17]:
# Original batch_size and num_steps
batch_size = 128
num_steps = 3001
# Restrict to only 5 possible batches
n_batches = 5

graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(1)  
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  # weights on the output layer
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  def nn_1layer(tf_dataset):
    '''
    The decision function for NN with 1 layer of hidden units
    '''
    # input layer
    input_vals = tf.matmul(tf_dataset, weights1) + biases1
    # hidden layer
    hidden_vals = tf.nn.relu(input_vals)
    # output layer 
    logits = tf.matmul(hidden_vals, weights2) + biases2
    return logits
  
  logits = nn_1layer(tf_train_dataset)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + 0.001*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(nn_1layer(tf_valid_dataset))
  test_prediction = tf.nn.softmax(nn_1layer(tf_test_dataset))

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Here I trick the offset to only having n_batches possible values to feed the model with 
    # 5 mini-batches over and over again
    batch_idx = step % n_batches
    offset = (batch_idx * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 640.594604
Minibatch accuracy: 6.2%
Validation accuracy: 26.0%
Minibatch loss at step 500: 191.008560
Minibatch accuracy: 100.0%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 115.837761
Minibatch accuracy: 100.0%
Validation accuracy: 75.4%
Minibatch loss at step 1500: 70.250420
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 2000: 42.603611
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 2500: 25.837515
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 3000: 15.670745
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Test accuracy: 83.7%


#### Conclusion for Problem2
The model is apparently overfitting in that train scores reached 100% and validation scores flattens. But I think the L2 regularization is still keeping the model from being more extremely overfitted.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [20]:
batch_size = 128
num_steps = 3001
# The probability that each element is kept
keep_prob = 0.9

graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(1)  
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  # weights on the output layer
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  def nn_1layer(tf_dataset, dropout=False):
    '''
    The decision function for NN with 1 layer of hidden units,
    if dropout is a float, applying dropout to on the hidden layer
    '''
    # input layer
    input_vals = tf.matmul(tf_dataset, weights1) + biases1
    # hidden layer
    hidden_vals = tf.nn.relu(input_vals)
    # drop out
    if dropout:
        hidden_vals = tf.nn.dropout(hidden_vals, tf.constant(dropout))
    # output layer 
    logits = tf.matmul(hidden_vals, weights2) + biases2
    return logits
  
  logits = nn_1layer(tf_train_dataset, dropout=keep_prob)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(nn_1layer(tf_valid_dataset))
  test_prediction = tf.nn.softmax(nn_1layer(tf_test_dataset))

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 339.332458
Minibatch accuracy: 10.2%
Validation accuracy: 26.4%
Minibatch loss at step 500: 19.650438
Minibatch accuracy: 82.0%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 10.208565
Minibatch accuracy: 77.3%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 5.836537
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 3.664526
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 3.051199
Minibatch accuracy: 80.5%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 2.945091
Minibatch accuracy: 80.5%
Validation accuracy: 82.4%
Test accuracy: 89.6%


In [21]:
batch_size = 128
num_steps = 3001
# Restrict to only 5 possible batches
n_batches = 5

# The probability that each element is kept
keep_prob = 0.9

graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(1)  
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  # weights on the output layer
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  def nn_1layer(tf_dataset, dropout=False):
    '''
    The decision function for NN with 1 layer of hidden units,
    if dropout is a float, applying dropout to on the hidden layer
    '''
    # input layer
    input_vals = tf.matmul(tf_dataset, weights1) + biases1
    # hidden layer
    hidden_vals = tf.nn.relu(input_vals)
    # drop out
    if dropout:
        hidden_vals = tf.nn.dropout(hidden_vals, tf.constant(dropout))
    # output layer 
    logits = tf.matmul(hidden_vals, weights2) + biases2
    return logits
  
  logits = nn_1layer(tf_train_dataset, dropout=keep_prob)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(nn_1layer(tf_valid_dataset))
  test_prediction = tf.nn.softmax(nn_1layer(tf_test_dataset))

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    batch_idx = step % n_batches
    offset = (batch_idx * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 339.332458
Minibatch accuracy: 10.2%
Validation accuracy: 26.4%
Minibatch loss at step 500: 0.194746
Minibatch accuracy: 99.2%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Test accuracy: 84.8%


In [23]:
batch_size = 128
num_steps = 3001
# Restrict to only 5 possible batches
n_batches = 5

# The probability that each element is kept
keep_prob = False

graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(1)  
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  # weights on the output layer
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  def nn_1layer(tf_dataset, dropout=False):
    '''
    The decision function for NN with 1 layer of hidden units,
    if dropout is a float, applying dropout to on the hidden layer
    '''
    # input layer
    input_vals = tf.matmul(tf_dataset, weights1) + biases1
    # hidden layer
    hidden_vals = tf.nn.relu(input_vals)
    # drop out
    if dropout:
        hidden_vals = tf.nn.dropout(hidden_vals, tf.constant(dropout))
    # output layer 
    logits = tf.matmul(hidden_vals, weights2) + biases2
    return logits
  
  logits = nn_1layer(tf_train_dataset, dropout=keep_prob)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(nn_1layer(tf_valid_dataset))
  test_prediction = tf.nn.softmax(nn_1layer(tf_test_dataset))

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    batch_idx = step % n_batches
    offset = (batch_idx * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 325.364380
Minibatch accuracy: 6.2%
Validation accuracy: 26.0%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Test accuracy: 83.2%


#### Conclusion for Problem3
Dropout can also alleviate overfitting

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [48]:
batch_size = 128
num_steps = 5001

# The probability that each element is kept
keep_prob = 1.0
n_hiddens1 = 1024
n_hiddens2 = 300
n_hiddens3 = 50

graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(1)  
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Init Variables
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hiddens1]))
  W2 = tf.Variable(tf.truncated_normal([n_hiddens1, n_hiddens2]))
  W3 = tf.Variable(tf.truncated_normal([n_hiddens2, n_hiddens3]))
  W4 = tf.Variable(tf.truncated_normal([n_hiddens3, num_labels]))
  B = tf.Variable(tf.zeros([num_labels]))
  
  def multi_layer_nn(X, keep_prob=False):
    # Write the computations
#     if keep_prob:
#         X = tf.nn.dropout(X, tf.constant(keep_prob))
    layer1 = tf.nn.tanh(tf.matmul(X, W1))
    if keep_prob:
        layer1 = tf.nn.dropout(layer1, tf.constant(keep_prob))
    layer2 = tf.nn.tanh(tf.matmul(layer1, W2))
    if keep_prob:
        layer2 = tf.nn.dropout(layer2, tf.constant(keep_prob))
#     logits = tf.matmul(layer2, W3) + B
    layer3 = tf.nn.tanh(tf.matmul(layer2, W3))
    if keep_prob:
        layer3 = tf.nn.dropout(layer3, tf.constant(keep_prob))
    logits = tf.matmul(layer3, W4) + B
    return logits

  logits = multi_layer_nn(tf_train_dataset, keep_prob=keep_prob)  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

#   global_step = tf.Variable(0)  # count the number of steps taken.
#   learning_rate = tf.train.exponential_decay(0.5, global_step, 50, 0.9)
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
#   optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  optimizer = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(loss)
    
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multi_layer_nn(tf_valid_dataset))
  test_prediction = tf.nn.softmax(multi_layer_nn(tf_test_dataset))

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    global_step += 1
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 8.973356
Minibatch accuracy: 8.6%
Validation accuracy: 10.4%
Minibatch loss at step 500: 0.918239
Minibatch accuracy: 71.1%
Validation accuracy: 67.6%
Minibatch loss at step 1000: 0.934827
Minibatch accuracy: 76.6%
Validation accuracy: 73.8%
Minibatch loss at step 1500: 0.695359
Minibatch accuracy: 75.8%
Validation accuracy: 74.0%
Minibatch loss at step 2000: 0.693429
Minibatch accuracy: 83.6%
Validation accuracy: 75.7%
Minibatch loss at step 2500: 0.621285
Minibatch accuracy: 82.0%
Validation accuracy: 76.8%
Minibatch loss at step 3000: 0.823607
Minibatch accuracy: 76.6%
Validation accuracy: 75.4%
Minibatch loss at step 3500: 0.810124
Minibatch accuracy: 80.5%
Validation accuracy: 77.8%
Minibatch loss at step 4000: 0.753341
Minibatch accuracy: 77.3%
Validation accuracy: 77.4%
Minibatch loss at step 4500: 0.686142
Minibatch accuracy: 78.1%
Validation accuracy: 78.5%
Minibatch loss at step 5000: 0.757969
Minibatch accuracy: 78.9%
Validation accuracy